This is the collection of codes that read food atlas datasets and CDC health indicator datasets from Github repository, integrate datasets and cleaning data

In [59]:
#merge food atlas datasets into one
import pandas as pd

Overall_folder='C:/Users/cathy/Capstone_project_1/'

dfs=list()
sub_folder='Datasets/Food_atlas/'
filenames=['ACCESS','ASSISTANCE','HEALTH','INSECURITY','LOCAL','PRICES_TAXES','RESTAURANTS','SOCIOECONOMIC','STORES']
for i,filename in enumerate(filenames):
    filepath=Overall_folder+sub_folder+filename+".csv"   
    d=pd.read_csv(filepath,index_col='FIPS')
    #append datasets to the list and drop the redundent columns:'State' and 'County'
    if i!=0:
        dfs.append(d.drop(['State', 'County'], axis=1))
    else:
        dfs.append(d)

#merge datasets
df_merge=pd.concat(dfs, join='outer', axis=1)

In [ ]:
print (df_merge.head(5))

Check columns for missing values

In [ ]:
df_merge.describe()

In [3]:
number_null_values_percol=df_merge.isnull().sum(axis=0)
#columns with over 100 missing values
cols_with_over_10_percent_null_values=number_null_values_percol[number_null_values_percol>314]
print (cols_with_over_10_percent_null_values.index)

Index(['PC_WIC_REDEMP08', 'PC_WIC_REDEMP12', 'PCH_PC_WIC_REDEMP_08_12',
       'REDEMP_WICS08', 'REDEMP_WICS12', 'PCH_REDEMP_WICS_08_12', 'PCT_HSPA15',
       'PCT_LOCLSALE07', 'PCH_DIRSALES_07_12', 'PCH_PC_DIRSALES_07_12',
       'PCH_FMRKT_09_16', 'PCH_FMRKTPTH_09_16', 'FMRKT_SNAP16',
       'PCT_FMRKT_SNAP16', 'FMRKT_WIC16', 'PCT_FMRKT_WIC16', 'FMRKT_WICCASH16',
       'PCT_FMRKT_WICCASH16', 'FMRKT_SFMNP16', 'PCT_FMRKT_SFMNP16',
       'FMRKT_CREDIT16', 'PCT_FMRKT_CREDIT16', 'FMRKT_FRVEG16',
       'PCT_FMRKT_FRVEG16', 'FMRKT_ANMLPROD16', 'PCT_FMRKT_ANMLPROD16',
       'FMRKT_BAKED16', 'PCT_FMRKT_BAKED16', 'FMRKT_OTHERFOOD16',
       'PCT_FMRKT_OTHERFOOD16', 'PCH_VEG_FARMS_07_12', 'VEG_ACRES07',
       'VEG_ACRES12', 'PCH_VEG_ACRES_07_12', 'VEG_ACRESPTH07',
       'VEG_ACRESPTH12', 'PCH_VEG_ACRESPTH_07_12', 'PCH_FRESHVEG_FARMS_07_12',
       'FRESHVEG_ACRES07', 'FRESHVEG_ACRES12', 'PCH_FRESHVEG_ACRES_07_12',
       'FRESHVEG_ACRESPTH07', 'FRESHVEG_ACRESPTH12',
       'PCH_FRESHVEG_A

In [4]:
#drop these columns first
df_merge=df_merge.drop(list(cols_with_over_10_percent_null_values.index), axis=1)

In [5]:
#check number of remaining columns
print (df_merge.columns)

Index(['State', 'County', 'LACCESS_POP10', 'LACCESS_POP15',
       'PCH_LACCESS_POP_10_15', 'PCT_LACCESS_POP10', 'PCT_LACCESS_POP15',
       'LACCESS_LOWI10', 'LACCESS_LOWI15', 'PCH_LACCESS_LOWI_10_15',
       ...
       'PCH_SNAPS_12_16', 'SNAPSPTH12', 'SNAPSPTH16', 'PCH_SNAPSPTH_12_16',
       'WICS08', 'WICS12', 'PCH_WICS_08_12', 'WICSPTH08', 'WICSPTH12',
       'PCH_WICSPTH_08_12'],
      dtype='object', length=209)


categorizes columns into three groups: category data ('State' and 'County'), count data, percent data, # per 1000 pop, and percent change

columns to keep: category data ('State' and 'County'), percent data, # per 1000 pop, and percent change; remove count data because it is not adjusted by population size

Each column name is highly abstract and unreadable, need to extract info from the variable information provided by Food_atlas

In [6]:
from collections import defaultdict

url='https://raw.githubusercontent.com/cathyxinxyz/Capstone_Project_1/master/Datasets/Food_atlas/variable_info.csv'
var_info_df=pd.read_csv(url,encoding="ISO-8859-1")
var_info_dict=defaultdict(list)
for idx in var_info_df.index:
    k=var_info_df['Units'][idx]
    var_info_dict[k].append(var_info_df['Variable Code'][idx])


further filter varaibles based on following principles:
i. keep variables that are adjusted by population size: '% change', 'Percent', '# per 1,000 pop','Percentage points';
ii. keep variables that are mostly valuable for analysis
iii. keep variables where values are valid: e.g. no negative values for variables with units as 'Percent' or '# per 1,000 pop'.


In [12]:
#units to keep: '% change', 'Percent', '# per 1,000 pop','Percentage points'
for k in var_info_dict.keys():
    if k not in ['Percent', '# per 1,000 pop','Dollars'] and var_info_dict[k] not in ['State','County']:
        df_merge = df_merge[df_merge.columns.difference(var_info_dict[k])]
        
#print (df_merge.shape)

In [13]:
#view variables
for idx in var_info_df.index:
    k=var_info_df['Units'][idx]
    k1=var_info_df['Category Code'][idx]
    k2=var_info_df['Sub_subcategory Name'][idx]
    var=var_info_df['Variable Code'][idx]
    
    if var in df_merge.columns:
        print (k1,k2,k,var)

ACCESS Overall Percent PCT_LACCESS_POP10
ACCESS Overall Percent PCT_LACCESS_POP15
ACCESS Low income Percent PCT_LACCESS_LOWI10
ACCESS Low income Percent PCT_LACCESS_LOWI15
ACCESS no car Percent PCT_LACCESS_HHNV10
ACCESS no car Percent PCT_LACCESS_HHNV15
ACCESS SNAP Percent PCT_LACCESS_SNAP15
ACCESS Children Percent PCT_LACCESS_CHILD10
ACCESS Children Percent PCT_LACCESS_CHILD15
ACCESS Seniors Percent PCT_LACCESS_SENIORS10
ACCESS Seniors Percent PCT_LACCESS_SENIORS15
ACCESS White Percent PCT_LACCESS_WHITE15
ACCESS Black Percent PCT_LACCESS_BLACK15
ACCESS Hispanic ethnicity Percent PCT_LACCESS_HISP15
ACCESS Asian Percent PCT_LACCESS_NHASIAN15
ACCESS American Indian or Alaska Native Percent PCT_LACCESS_NHNA15
ACCESS Hawaiian or Pacific Islander Percent PCT_LACCESS_NHPI15
ACCESS Multiracial Percent PCT_LACCESS_MULTIR15
STORES Grocery # per 1,000 pop GROCPTH09
STORES Grocery # per 1,000 pop GROCPTH14
STORES Supercenters # per 1,000 pop SUPERCPTH09
STORES Supercenters # per 1,000 pop SUPERCP

In [14]:
print (df_merge.shape)

(3143, 94)


In [15]:
print(df_merge.shape)
#view variables
for idx in var_info_df.index:
    k=var_info_df['Units'][idx]
    k1=var_info_df['Category Code'][idx]
    k2=var_info_df['Sub_subcategory Name'][idx]
    var=var_info_df['Variable Code'][idx]
    
    if var in df_merge.columns:
        print (k1,k2,k,var)

(3143, 94)
ACCESS Overall Percent PCT_LACCESS_POP10
ACCESS Overall Percent PCT_LACCESS_POP15
ACCESS Low income Percent PCT_LACCESS_LOWI10
ACCESS Low income Percent PCT_LACCESS_LOWI15
ACCESS no car Percent PCT_LACCESS_HHNV10
ACCESS no car Percent PCT_LACCESS_HHNV15
ACCESS SNAP Percent PCT_LACCESS_SNAP15
ACCESS Children Percent PCT_LACCESS_CHILD10
ACCESS Children Percent PCT_LACCESS_CHILD15
ACCESS Seniors Percent PCT_LACCESS_SENIORS10
ACCESS Seniors Percent PCT_LACCESS_SENIORS15
ACCESS White Percent PCT_LACCESS_WHITE15
ACCESS Black Percent PCT_LACCESS_BLACK15
ACCESS Hispanic ethnicity Percent PCT_LACCESS_HISP15
ACCESS Asian Percent PCT_LACCESS_NHASIAN15
ACCESS American Indian or Alaska Native Percent PCT_LACCESS_NHNA15
ACCESS Hawaiian or Pacific Islander Percent PCT_LACCESS_NHPI15
ACCESS Multiracial Percent PCT_LACCESS_MULTIR15
STORES Grocery # per 1,000 pop GROCPTH09
STORES Grocery # per 1,000 pop GROCPTH14
STORES Supercenters # per 1,000 pop SUPERCPTH09
STORES Supercenters # per 1,000 

In [16]:
#devide dataframe into two: one with variables measured at one year and one with variables as percent change、
var_timepoint=list()
var_percentchange=list()
for idx in var_info_df.index:
    k=var_info_df['Units'][idx]
    k1=var_info_df['Category Code'][idx]
    k2=var_info_df['Sub_subcategory Name'][idx]
    var=var_info_df['Variable Code'][idx]
    
    if var in df_merge.columns: 
        var_timepoint.append(var)

var_timepoint.extend(['State','County'])
var_percentchange.extend(['State','County'])
df_tp=df_merge[var_timepoint]


In [18]:
print (df_tp.shape)
print (df_pr.shape)

(3143, 49)
(3143, 18)
Index(['PCH_LACCESS_POP_10_15', 'PCH_LACCESS_LOWI_10_15',
       'PCH_LACCESS_SENIORS_10_15', 'PCH_GROC_09_14', 'PCH_CONVS_09_14',
       'PCH_CONVSPTH_09_14', 'PCH_SNAPS_12_16', 'PCH_SNAPSPTH_12_16',
       'PCH_WICS_08_12', 'PCH_WICSPTH_08_12', 'PCH_FFR_09_14',
       'PCH_FFRPTH_09_14', 'PCH_FSR_09_14', 'PCH_CACFP_09_15',
       'CH_FOODINSEC_12_15', 'CH_VLFOODSEC_12_15', 'State', 'County'],
      dtype='object')


In [18]:
#check weather each column has valid values:
####### columns with units 'Percent' should have values between 0 and 100, any value that fall out of this range should be changed to NaN values
###### 
######
######

#Replace invalid values with np.nan
import numpy as np

for idx in var_info_df.index:
    k=var_info_df['Units'][idx]
    k1=var_info_df['Category Code'][idx]
    k2=var_info_df['Sub_subcategory Name'][idx]
    var=var_info_df['Variable Code'][idx]
    
    if k =='Percent' and var in df_tp.columns: 
        ser=df_tp[var]<0 
        ser=ser+df_tp[var]>100
        if ser.sum()>0:
            print ((k1,k2,var,ser.sum()))
            df_tp[var][(df_tp[var]<0)|(df_tp[var]>100)]=np.nan
    elif k=='# per 1,000 pop' and var in df_tp.columns:
        ser=df_tp[var]<0
        ser=ser+df_tp[var]>1000
        if ser.sum()>0:
            print ((k1,k2,var,ser.sum()))
            df_tp[var][(df_tp[var]<0)|(df_tp[var]>1000)]=np.nan


        

In [34]:
#break df_tp into two sets: variables measured at the earlier time point: df_tp_1; and variables measured at the earlier time point: df_tp_2

#group the same measure into tuples, the same measure share the same name except the last two digits which indicate the year of the measure
var_grouped_by_measures=defaultdict(list)
early_measure_list=['State','County']
late_measure_list=['State','County']
for idx in var_info_df.index:
    k=var_info_df['Units'][idx]
    k1=var_info_df['Category Code'][idx]
    k2=var_info_df['Sub_subcategory Name'][idx]
    var=var_info_df['Variable Code'][idx]
    
    if var in df_tp.columns and var not in ['State','County']:
        var_grouped_by_measures[(k1,k2)].append((var, float(var[-2:])))
    
for v in var_grouped_by_measures.values():
    v.sort(key=lambda tup: tup[-1])
    if float(v[-1][0][-2:])>13 and float(v[0][0][-2:])<13:
        late_measure_list.append(v[0][0])
    elif float(v[-1][0][-2:])<13:
        late_measure_list.append(v[-1][0])


df_tp_later=df_tp[late_measure_list]

In [35]:
print (df_tp_later.shape)

(3143, 41)


In [36]:
print (df_tp_later.columns)

Index(['State', 'County', 'PCT_LACCESS_POP10', 'PCT_LACCESS_LOWI10',
       'PCT_LACCESS_HHNV10', 'PCT_LACCESS_CHILD10', 'PCT_LACCESS_SENIORS10',
       'GROCPTH09', 'SUPERCPTH09', 'CONVSPTH09', 'SPECSPTH09', 'SNAPSPTH12',
       'WICSPTH12', 'FFRPTH09', 'FSRPTH09', 'PC_FFRSALES12', 'PC_FSRSALES12',
       'PCT_SNAP12', 'PCT_NSLP09', 'PCT_FREE_LUNCH09', 'PCT_REDUCED_LUNCH09',
       'PCT_SBP09', 'PCT_SFSP09', 'PCT_WIC09', 'PCT_CACFP09',
       'FOODINSEC_10_12', 'VLFOODSEC_10_12', 'FOODINSEC_CHILD_03_11',
       'PCT_LOCLFARM12', 'PCT_LOCLSALE12', 'PC_DIRSALES12', 'FMRKTPTH09',
       'RECFACPTH09', 'PCT_NHWHITE10', 'PCT_NHBLACK10', 'PCT_HISP10',
       'PCT_NHASIAN10', 'PCT_NHNA10', 'PCT_NHPI10', 'PCT_65OLDER10',
       'PCT_18YOUNGER10'],
      dtype='object')


In [41]:
print (df_tp_later.head(5))

     State   County  PCT_LACCESS_POP10  PCT_LACCESS_LOWI10  \
FIPS                                                         
1001    AL  Autauga          33.769657            9.793530   
1003    AL  Baldwin          19.318473            5.460261   
1005    AL  Barbour          20.840972           11.420316   
1007    AL     Bibb           4.559753            2.144661   
1009    AL   Blount           2.700840            1.062468   

      PCT_LACCESS_HHNV10  PCT_LACCESS_CHILD10  PCT_LACCESS_SENIORS10  \
FIPS                                                                   
1001            3.284786             8.837112               4.376378   
1003            2.147827             4.343199               3.513570   
1005            4.135869             3.425062               2.805166   
1007            3.458580             1.087518               0.657008   
1009            3.269380             0.671490               0.340269   

      GROCPTH09  SUPERCPTH09  CONVSPTH09       ...         F

In [37]:
df_tp_later.to_csv(Overall_folder+'Datasets/Food_atlas/df_tp_new.csv')

Integrate CDC Datasets together

In [25]:
import pandas as pd
dfs=list()
sub_folder=Overall_folder+'/Datasets/CDC/'
filenames=['Diabetes_prevalence',
           'Obesity_prevalence',
           'Physical_inactive_prevalence']

In [26]:
for filename in filenames:
    filepath=sub_folder+filename+".csv"   
    df=pd.read_csv(filepath,index_col='FIPS')
    
    
    if 'Diabetes' in filename:
        df.columns=df.columns.astype(str)+'_db'
    elif 'Obesity' in filename:
        df.columns=df.columns.astype(str)+'_ob'
    elif 'Physical' in filename:
        df.columns=df.columns.astype(str)+'_phy'
    dfs.append(df)
#merge datasets


In [27]:
CDC_merge=pd.concat(dfs, join='outer', axis=1)

In [28]:
CDC_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3224 entries, 1001 to 72153
Data columns (total 30 columns):
2004_db     3224 non-null object
2005_db     3224 non-null object
2006_db     3224 non-null object
2007_db     3224 non-null object
2008_db     3224 non-null object
2009_db     3224 non-null object
2010_db     3224 non-null object
2011_db     3224 non-null object
2012_db     3224 non-null object
2013_db     3224 non-null object
2004_ob     3224 non-null object
2005_ob     3224 non-null object
2006_ob     3224 non-null object
2007_ob     3224 non-null object
2008_ob     3224 non-null object
2009_ob     3224 non-null object
2010_ob     3224 non-null object
2011_ob     3146 non-null object
2012_ob     3146 non-null object
2013_ob     3146 non-null object
2004_phy    3224 non-null object
2005_phy    3224 non-null object
2006_phy    3224 non-null object
2007_phy    3224 non-null object
2008_phy    3224 non-null object
2009_phy    3224 non-null object
2010_phy    3224 non-null objec

In [29]:
#Find out the non numeric entries in CDC_merge
for c in CDC_merge.columns:
    num_non_numeric=sum(CDC_merge.applymap(lambda x: isinstance(x, (int, float)))[c])
    if num_non_numeric>0:
        print(c, num_non_numeric, CDC_merge[pd.to_numeric(CDC_merge[c], errors='coerce').isnull()])

2011_ob 78       2004_db             2005_db 2006_db 2007_db 2008_db  2009_db  2010_db  \
FIPS                                                                          
2201      5.8  6.1000000000000005     5.9     6.8     7.4  No Data  No Data   
2232      5.8  6.1000000000000005     6.6     6.9     5.2  No Data  No Data   
2280      5.4                 6.2     6.4       7       7  No Data  No Data   
72001    11.4                10.9    12.8    12.7    14.3     14.7     15.4   
72003    12.4                14.9    16.3    15.5    14.7     14.6     16.4   
72005    12.8                14.6    16.3    14.9    15.4       15     15.9   
72007    11.1                12.9      14    14.7    14.7     15.1       15   
72009      13                13.6    13.7    13.2    13.2     12.5     13.9   
72011    12.1                13.8    15.3    14.9    14.7     14.8     16.9   
72013    13.8                14.5    14.8    15.3    16.4     17.9     17.2   
72015    13.6                13.9    13.4

[82 rows x 30 columns]
2012_phy 78       2004_db             2005_db 2006_db 2007_db 2008_db  2009_db  2010_db  \
FIPS                                                                          
2201      5.8  6.1000000000000005     5.9     6.8     7.4  No Data  No Data   
2232      5.8  6.1000000000000005     6.6     6.9     5.2  No Data  No Data   
2280      5.4                 6.2     6.4       7       7  No Data  No Data   
72001    11.4                10.9    12.8    12.7    14.3     14.7     15.4   
72003    12.4                14.9    16.3    15.5    14.7     14.6     16.4   
72005    12.8                14.6    16.3    14.9    15.4       15     15.9   
72007    11.1                12.9      14    14.7    14.7     15.1       15   
72009      13                13.6    13.7    13.2    13.2     12.5     13.9   
72011    12.1                13.8    15.3    14.9    14.7     14.8     16.9   
72013    13.8                14.5    14.8    15.3    16.4     17.9     17.2   
72015    13.6    

In [30]:
#It turns out that some entries are 'No Data' or NaN, so I replace the 'No Data' with NaN values
CDC_merge=CDC_merge.replace('No Data', np.nan)
CDC_merge=CDC_merge.astype(float)

In [31]:
#now check the CDC_merge
CDC_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3224 entries, 1001 to 72153
Data columns (total 30 columns):
2004_db     3219 non-null float64
2005_db     3219 non-null float64
2006_db     3219 non-null float64
2007_db     3219 non-null float64
2008_db     3219 non-null float64
2009_db     3221 non-null float64
2010_db     3221 non-null float64
2011_db     3221 non-null float64
2012_db     3221 non-null float64
2013_db     3220 non-null float64
2004_ob     3141 non-null float64
2005_ob     3141 non-null float64
2006_ob     3141 non-null float64
2007_ob     3141 non-null float64
2008_ob     3141 non-null float64
2009_ob     3143 non-null float64
2010_ob     3143 non-null float64
2011_ob     3143 non-null float64
2012_ob     3143 non-null float64
2013_ob     3142 non-null float64
2004_phy    3141 non-null float64
2005_phy    3141 non-null float64
2006_phy    3141 non-null float64
2007_phy    3141 non-null float64
2008_phy    3141 non-null float64
2009_phy    3143 non-null float64
2010_

In [38]:
#choose the latest prevalence of diabetes, obesity and physical inactivity to merge with df_tp 
combined=pd.concat([df_tp_later, CDC_merge[['2013_db','2013_ob','2013_phy']]], join='inner',axis=1)

In [39]:
combined.rename(columns={'2013_db': 'prevalence of diabetes', '2013_ob': 'prevalence of obesity', '2013_phy':'prevalence of physical inactivity'}, inplace=True)

Integrating geography dataset

In [63]:
df_rbcodes=pd.read_csv(Overall_folder+'Datasets/geography/FIPS_RUCC_Code_2013.csv', index_col='FIPS')

In [64]:
df_rbcodes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3221 entries, 1001 to 72153
Data columns (total 2 columns):
Unnamed: 0    3221 non-null int64
RUCC_2013     3221 non-null int64
dtypes: int64(2)
memory usage: 75.5 KB


In [66]:
combined=pd.concat([combined, df_rbcodes[['RUCC_2013']]], join='inner',axis=1)

In [68]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3143 entries, 1001 to 56045
Data columns (total 45 columns):
State                                3143 non-null object
County                               3143 non-null object
PCT_LACCESS_POP10                    3118 non-null float64
PCT_LACCESS_LOWI10                   3143 non-null float64
PCT_LACCESS_HHNV10                   3143 non-null float64
PCT_LACCESS_CHILD10                  3143 non-null float64
PCT_LACCESS_SENIORS10                3143 non-null float64
GROCPTH09                            3143 non-null float64
SUPERCPTH09                          3143 non-null float64
CONVSPTH09                           3143 non-null float64
SPECSPTH09                           3143 non-null float64
SNAPSPTH12                           3143 non-null float64
WICSPTH12                            3143 non-null float64
FFRPTH09                             3143 non-null float64
FSRPTH09                             3143 non-null float64
PC_FFR

In [70]:
df_divisions=pd.read_csv(Overall_folder+'Datasets/geography/state_region.csv')

In [71]:
df_divisions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 5 columns):
Unnamed: 0    51 non-null int64
State         51 non-null object
State Code    51 non-null object
Region        51 non-null object
Division      51 non-null object
dtypes: int64(1), object(4)
memory usage: 2.1+ KB


In [73]:
division_dict=df_divisions.set_index('State Code').to_dict()

In [77]:
division_dict

{'Division': {'AK': 'Pacific',
  'AL': 'East South Central',
  'AR': 'West South Central',
  'AZ': 'Mountain',
  'CA': 'Pacific',
  'CO': 'Mountain',
  'CT': 'New England',
  'DC': 'South Atlantic',
  'DE': 'South Atlantic',
  'FL': 'South Atlantic',
  'GA': 'South Atlantic',
  'HI': 'Pacific',
  'IA': 'West North Central',
  'ID': 'Mountain',
  'IL': 'East North Central',
  'IN': 'East North Central',
  'KS': 'West North Central',
  'KY': 'East South Central',
  'LA': 'West South Central',
  'MA': 'New England',
  'MD': 'South Atlantic',
  'ME': 'New England',
  'MI': 'East North Central',
  'MN': 'West North Central',
  'MO': 'West North Central',
  'MS': 'East South Central',
  'MT': 'Mountain',
  'NC': 'South Atlantic',
  'ND': 'West North Central',
  'NE': 'West North Central',
  'NH': 'New England',
  'NJ': 'Middle Atlantic',
  'NM': 'Mountain',
  'NV': 'Mountain',
  'NY': 'Middle Atlantic',
  'OH': 'East North Central',
  'OK': 'West South Central',
  'OR': 'Pacific',
  'PA': 'M

In [78]:
combined['Division']=combined['State'].map(division_dict['Division'])
combined['Region']=combined['State'].map(division_dict['Region'])

In [79]:
combined.head(5)

,State,County,PCT_LACCESS_POP10,PCT_LACCESS_LOWI10,PCT_LACCESS_HHNV10,PCT_LACCESS_CHILD10,PCT_LACCESS_SENIORS10,GROCPTH09,SUPERCPTH09,CONVSPTH09,...,PCT_NHNA10,PCT_NHPI10,PCT_65OLDER10,PCT_18YOUNGER10,prevalence of diabetes,prevalence of obesity,prevalence of physical inactivity,RUCC_2013,Division,Region
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,AL,Autauga,33.769657,9.793530,3.284786,8.837112,4.376378,0.110834,0.018472,0.535698,...,0.397647,0.040314,11.995382,26.777959,13.0,34.1,28.6,2,East South Central,South
1003,AL,Baldwin,19.318473,5.460261,2.147827,4.343199,3.513570,0.133775,0.033444,0.663300,...,0.628755,0.043343,16.771185,22.987408,10.4,27.4,22.3,3,East South Central,South
1005,AL,Barbour,20.840972,11.420316,4.135869,3.425062,2.805166,0.180786,0.000000,0.506201,...,0.218524,0.087409,14.236807,21.906982,18.4,44.4,31.8,6,East South Central,South
1007,AL,Bibb,4.559753,2.144661,3.458580,1.087518,0.657008,0.261540,0.043590,0.828211,...,0.279293,0.030548,12.681650,22.696923,14.8,40.3,33.9,1,East South Central,South
1009,AL,Blount,2.700840,1.062468,3.269380,0.671490,0.340269,0.104637,0.017440,0.540625,...,0.497191,0.031402,14.722096,24.608353,14.1,34.6,28.0,1,East South Central,South


In [80]:
combined.to_csv(Overall_folder+'Datasets/Combined_data_new.csv')